In [ ]:
%load_ext lab_black

In [ ]:
# https://sql.sh/736-base-donnees-villes-francaises

In [ ]:
villes = pd.read_csv(
    "data/villes_france.csv",
    usecols=[5, 8, 10, 16, 19, 20, 25, 26],
    names=[
        "nom",
        "code_postal",
        "code",
        "population",
        "longitude",
        "latitude",
        "altitude_min",
        "altitude_max",
    ],
    dtype={"code_postal": str, "code": str},
).query('~code_postal.str.startswith("97")')

In [ ]:
import geopandas as gpd

github_url = "https://raw.githubusercontent.com/{user}/{repo}/master/{path}"

departements_fr = gpd.GeoDataFrame.from_file(
    github_url.format(
        user="gregoiredavid",
        repo="france-geojson",
        path="departements-version-simplifiee.geojson",
    )
)

communes_fr = gpd.GeoDataFrame.from_file(
    github_url.format(
        user="gregoiredavid",
        repo="france-geojson",
        path="communes-version-simplifiee.geojson",
    )
)

com = communes_fr.merge(villes, on="code")

alt.data_transformers.disable_max_rows()


In [ ]:
chart = (
    alt.hconcat(
        alt.layer(
            alt.Chart(departements_fr)
            .mark_geoshape(fill="none", stroke="#bab0ac", strokeWidth=0.5)
            .project("conicConformal", rotate=[-3, -46.5], parallels=[49, 44]),
            alt.Chart(villes)
            .mark_circle()
            .encode(
                alt.Latitude("latitude"),
                alt.Longitude("longitude"),
                alt.Size("population", title="Population"),
            ),
        ).properties(width=300, height=300),
        alt.Chart(com)
        .mark_geoshape()
        .encode(
            alt.Color("population", title="Population", scale=alt.Scale(type="symlog"))
        )
        .project("conicConformal", rotate=[-3, -46.5], parallels=[49, 44])
        .properties(width=300, height=300),
    )
    .configure_view(stroke=None)
    .configure_legend(
        orient="bottom",
        titleFontSize=14,
        labelFontSize=13,
        titleFont="Fira Sans",
        labelFont="Ubuntu",
    )
)
chart

In [ ]:
chart.save("challenge_day12.svg")